# SQL

## test

In [2]:
# loading in modules
import sqlite3

#file path
dbfile = 'Database/olist.db'
# SQL connection
con = sqlite3.connect(dbfile)

# creating cursor
cur = con.cursor()

# Test
table_list = [a for a in cur.execute("SELECT name FROM sqlite_master WHERE type = 'table'")]
print(table_list)

# close connection
con.close()

[('customers',), ('geoloc',), ('order_items',), ('order_pymts',), ('order_reviews',), ('orders',), ('products',), ('sellers',), ('translation',)]


## Requêtes

### !!! vérification dates !!!

In [10]:
# Vérifier les dates des commandes dans les 3 derniers mois
query_check_dates = """
SELECT 
    order_id, 
    order_purchase_timestamp 
FROM 
    orders 
WHERE 
    julianday('now') - julianday(order_purchase_timestamp) <= 90
LIMIT 5;  -- Limite à 5 pour inspecter les résultats
"""

# Exécution de la requête et affichage des résultats
df_check_dates = pd.read_sql_query(query_check_dates, conn)
print("Exemples de commandes des 3 derniers mois :")
print(df_check_dates)


Exemples de commandes des 3 derniers mois :
Empty DataFrame
Columns: [order_id, order_purchase_timestamp]
Index: []


In [14]:
# Vérifier les dates de la première et de la dernière commande dans la base de données
query_check_date_range = """
SELECT 
    MIN(order_purchase_timestamp) AS first_order_date,
    MAX(order_purchase_timestamp) AS last_order_date
FROM 
    orders;
"""

# Exécution de la requête et affichage des résultats
df_check_date_range = pd.read_sql_query(query_check_date_range, conn)
print("Plage de dates des commandes dans la base de données :")
print(df_check_date_range)


Plage de dates des commandes dans la base de données :
      first_order_date      last_order_date
0  2016-09-04 21:15:19  2018-10-17 17:30:18


### Sur date cohérente

In [24]:
import sqlite3
import pandas as pd
from tabulate import tabulate

# Connexion à la base de données
conn = sqlite3.connect('Database/olist.db')

# Requête 1 : Commandes récentes reçues avec au moins 3 jours de retard (delay_in_days arrondi sans virgule)
query_1 = """
SELECT 
    o.order_id, 
    o.order_purchase_timestamp, 
    o.order_delivered_customer_date, 
    ROUND(julianday(o.order_delivered_customer_date) - julianday(o.order_estimated_delivery_date)) AS delay_in_days
FROM 
    orders o
WHERE 
    o.order_status != 'canceled' 
    AND julianday(o.order_purchase_timestamp) >= julianday('2018-07-17')  -- Limiter aux 3 derniers mois de données
    AND ROUND(julianday(o.order_delivered_customer_date) - julianday(o.order_estimated_delivery_date)) > 3;
"""

# Requête 2 : Vendeurs ayant généré un chiffre d'affaires de plus de 100 000 R$ via Olist
query_2 = """
SELECT 
    oi.seller_id, 
    SUM(oi.price) AS total_revenue
FROM 
    order_items oi
JOIN 
    orders o ON oi.order_id = o.order_id
WHERE 
    o.order_status = 'delivered'
GROUP BY 
    oi.seller_id
HAVING 
    total_revenue > 100000;
"""

# Requête 3 : Nouveaux vendeurs très engagés (moins de 3 mois d'ancienneté, ayant vendu plus de 30 produits)
query_3 = """
SELECT 
    oi.seller_id, 
    COUNT(oi.order_id) AS total_products_sold,
    MIN(o.order_purchase_timestamp) AS first_order_date
FROM 
    order_items oi
JOIN 
    orders o ON oi.order_id = o.order_id
WHERE 
    julianday('2018-10-17') - julianday(o.order_purchase_timestamp) <= 90  -- Utiliser la période de juillet à octobre 2018
GROUP BY 
    oi.seller_id
HAVING 
    total_products_sold > 30;
"""

# Requête 4 : Les 5 codes postaux avec plus de 30 reviews et le pire score moyen de review sur les 12 derniers mois
query_4 = """
SELECT 
    c.customer_zip_code_prefix, 
    COUNT(r.review_id) AS total_reviews, 
    AVG(r.review_score) AS avg_review_score
FROM 
    orders o
JOIN 
    order_reviews r ON o.order_id = r.order_id
JOIN 
    customers c ON o.customer_id = c.customer_id
WHERE 
    julianday('2018-10-17') - julianday(o.order_purchase_timestamp) <= 365  -- Les 12 derniers mois de données
GROUP BY 
    c.customer_zip_code_prefix
HAVING 
    total_reviews > 30
ORDER BY 
    avg_review_score ASC
LIMIT 5;
"""

# Exécution des requêtes et stockage des résultats dans des DataFrames
df_recent_orders_with_delay = pd.read_sql_query(query_1, conn)
df_high_revenue_sellers = pd.read_sql_query(query_2, conn)
df_new_engaged_sellers = pd.read_sql_query(query_3, conn)
df_worst_reviewed_zip_codes = pd.read_sql_query(query_4, conn)

# Fermeture de la connexion à la base de données
conn.close()

# Affichage des résultats avec tabulate
print("Commandes récentes reçues avec au moins 3 jours de retard au cours des 3 derniers mois:")
print(tabulate(df_recent_orders_with_delay, headers='keys', tablefmt='pipe'))

print("\nVendeurs ayant généré un chiffre d'affaires de plus de 100 000 R$ via Olist:")
print(tabulate(df_high_revenue_sellers, headers='keys', tablefmt='pipe'))

print("\nNouveaux vendeurs très engagés (moins de 3 mois d'ancienneté, ayant vendu plus de 30 produits):")
print(tabulate(df_new_engaged_sellers, headers='keys', tablefmt='pipe'))

print("\nLes 5 codes postaux avec plus de 30 reviews et le pire score moyen de review sur les 12 derniers mois:")
print(tabulate(df_worst_reviewed_zip_codes, headers='keys', tablefmt='pipe'))


Commandes récentes reçues avec au moins 3 jours de retard:
|     | order_id                         | order_purchase_timestamp   | order_delivered_customer_date   |   delay_in_days |
|----:|:---------------------------------|:---------------------------|:--------------------------------|----------------:|
|   0 | cfa4fa27b417971e86d8127cb688712f | 2018-08-16 09:44:23        | 2018-08-29 01:41:41             |               7 |
|   1 | 234c056c50619f48da64f731c48242b4 | 2018-08-14 14:49:15        | 2018-09-01 18:14:42             |              10 |
|   2 | 8ad3f1d0f96992e43566c4c82c9f6c58 | 2018-07-17 21:25:29        | 2018-08-14 04:04:40             |              11 |
|   3 | 7f579e203c931f3e8410103359c6d523 | 2018-08-02 18:09:27        | 2018-08-13 20:11:47             |               5 |
|   4 | 03720fdc92032ee4abd471d172006ab0 | 2018-08-05 21:34:54        | 2018-08-21 00:11:52             |               4 |
|   5 | 7a268da1c6173cf3d0847a89afdaf84e | 2018-07-23 19:10:00        | 2

## Export

In [38]:
# Connexion à la base de données
conn = sqlite3.connect('Database/olist.db')

# Obtenir les noms des tables
tables = conn.execute("SELECT name FROM sqlite_master WHERE type='table';").fetchall()
table_names = [table[0] for table in tables]

# Exporter chaque table en un fichier CSV
for table in table_names:
    df = pd.read_sql_query(f"SELECT * FROM {table}", conn)
    df.to_csv(f'Database/{table}.csv', index=False)
    print(f"Table {table} exportée avec succès dans le fichier Database/{table}.csv")

# Fermeture de la connexion à la base de données
conn.close()

print("Toutes les tables ont été exportées avec succès.")


Table customers exportée avec succès dans le fichier Database/customers.csv
Table geoloc exportée avec succès dans le fichier Database/geoloc.csv
Table order_items exportée avec succès dans le fichier Database/order_items.csv
Table order_pymts exportée avec succès dans le fichier Database/order_pymts.csv
Table order_reviews exportée avec succès dans le fichier Database/order_reviews.csv
Table orders exportée avec succès dans le fichier Database/orders.csv
Table products exportée avec succès dans le fichier Database/products.csv
Table sellers exportée avec succès dans le fichier Database/sellers.csv
Table translation exportée avec succès dans le fichier Database/translation.csv
Toutes les tables ont été exportées avec succès.
